<a href="https://colab.research.google.com/github/ericmcg13/CCT-CA1/blob/main/CA1_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

'Dash' is a required install for use of the plotly graphics and sliders.

'ipyleaflet' is used for the geo mapping and Choropleth layer



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWghgqs3iUrzgCaaWPITgCfzP4IXKRXJ7r8zND24pF8jonGq7ZZfE8g


# New section

In [76]:
!pip install dash
!pip install ipyleaflet

     |████████████████████████████████| 7.3 MB 5.1 MB/s 
     |████████████████████████████████| 25.3 MB 1.3 MB/s 
     |████████████████████████████████| 357 kB 50.5 MB/s 
  Created wheel for dash-core-components: filename=dash_core_components-2.0.0-py3-none-any.whl size=3821 sha256=ee83edd2ecc3e70ef09fa336717156f8497313553bd5bea8bcce0dae9e173b16
  Stored in directory: /root/.cache/pip/wheels/00/f9/c7/1a6437d794ed753ea9bc9079e761d4fc803a1f1f5d3697b9ec
  Created wheel for dash-html-components: filename=dash_html_components-2.0.0-py3-none-any.whl size=4089 sha256=332204681bbac50583eb91dba31c17a11f14319c6bf0c94988a8fed53bf76a17
  Stored in directory: /root/.cache/pip/wheels/ec/6b/81/05aceabd8b27f724e2c96784016287cc1bfbc349ebfda451de
  Created wheel for dash-table: filename=dash_table-5.0.0-py3-none-any.whl size=3911 sha256=ef0784ede9f7f4f0beb47eb570b36e7ce94e6fd2ddc79554109fcdc18f343c06
  Stored in directory: /root/.cache/pip/wheels/85/5d/4e/7c276b57992951dbe770bf5caad6448d0539c510663aef

     |████████████████████████████████| 3.3 MB 4.9 MB/s 


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import numpy as np

# Set the plot to retina quality
%config InlineBackend.figure_format = "retina"

# Set the 'talk' library in SNS (Changes the size and labels to graphs to make them bigger)
sns.set_context('talk')

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Multiple cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_mode_interactivity = 'all'

###

# Below is only require for COLAB

###
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
migration_df = pd.read_csv("/content/drive/MyDrive/Migration.csv")
migration_df.head(5)

In [ ]:
migration_df.shape
migration_df.info()

In [ ]:
# Remove NaN values
migration_df.dropna(inplace=True)

In [ ]:
migration_df.info()

In [ ]:
# Group the totals by age cohort
migration_sum_df = migration_df.groupby('Age Group').sum()

# Remove the TOTAL column from the graph as it distorts the figures
# migration_sum_df.drop("All ages", axis=0, inplace=True)
migration_sum_df

In [ ]:
# Filter the df by NET MIGRATION for ALL AGES and BOTH SEXES to get the totals
net_migration = migration_df[(migration_df['Inward or Outward Flow'] == 'Net migration') & 
                             (migration_df['Age Group'] == 'All ages') &
                             (migration_df['Sex'] == 'Both sexes')]
net_migration.head()

In [ ]:
# Show the inital graph to get a sense of the pattern (if any)
fig = px.line(net_migration, x='Year', y='VALUE', color='Sex')
fig.show()

In [ ]:
# Bring in the Birth/Death dataframe
bdm_df = pd.read_csv('/content/drive/MyDrive/BDM_Ireland1960_2021.csv')
bdm_df.head()

In [ ]:
bdm_df.info()
bdm_df.shape

In [ ]:
# Computer the net delta population growth (birth - death)
bdm_df['Growth'] = bdm_df['Births Registered (Number)'] - bdm_df['Deaths Registered (Number)']

# Use Lambda to iterate over the values in each cell to remove the QUARTER time values.
bdm_df['Quarter'] = bdm_df['Quarter'].map(lambda x: str(x)[:-2])

In [ ]:
bdm_df.head()

In [ ]:
# We only need the time and growth delta for graphing
growth = bdm_df[['Quarter', 'Growth']].copy()
growth = growth.groupby(['Quarter'], as_index=False)['Growth'].sum()

# Quarter has been removed, only YEAR remains
growth.columns = ['Year', 'Growth Delta']
growth.head()

In [ ]:
# drop years less than 1987 for comparison with Growth
compare_growth = growth[growth['Year'] > '1986']

fig1 = px.line(compare_growth, x='Year', y='Growth Delta')
fig1.show()

In [ ]:
# We only need the Year and migration totals for graphing
migration_df_simplified = net_migration[['Year', 'VALUE']].copy()

# Convert STRING year in compare_growth to INT so that a JOIN can be performed
compare_growth['Year'] = pd.to_numeric(compare_growth['Year'])

# Join Columns
merge_df = pd.merge(compare_growth, migration_df_simplified, how='outer', on='Year')

# Fix VALUE column name and add
merge_df.rename({'VALUE':'Migration Delta'}, axis=1, inplace=True)
merge_df['Migration Delta'] = merge_df['Migration Delta'].apply(lambda x: int(x * 1000))
merge_df.head()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig2 = make_subplots(specs=[[{"secondary_y": True}]])

# Add trace - Growth
fig2.add_trace(
    go.Scatter(x=merge_df['Year'], y=merge_df['Growth Delta'], name="Growth"),
    secondary_y=False,
)

# Add trace - Migration
fig2.add_trace(
    go.Scatter(x=merge_df['Year'], y=merge_df['Migration Delta'], name="Migration"),
    secondary_y=False,
)

# Add figure title
fig2.update_layout(
    title_text="Pop Growth and Migration"
)

# Set x-axis title
fig2.update_xaxes(title_text="Year")

# Set y-axes titles
fig2.update_yaxes(title_text="Population", secondary_y=False)
fig2.show()

In [ ]:
# Import the employment data
employment_df = pd.read_csv('/content/drive/MyDrive/Vacancies.csv')
employment_df.head()

In [ ]:
employment_df.info()
employment_df.shape

In [ ]:
employment_df['Statistic'].unique()

In [ ]:
# We need the total number of vacancies for all sectors in all industries to get the totals
employment_df = employment_df[(employment_df['Statistic'] == 'Number of Job Vacancies') &
                             (employment_df['Private or Public Sector'] == 'All sectors') &
                             (employment_df['Economic Sector NACE Rev 2'] == 'All NACE economic sectors')] 

employment_df.head()

In [ ]:
# Use Lambda to iterate over the values in each cell and remove QUARTER time data
employment_df['Quarter'] = employment_df['Quarter'].map(lambda x: str(x)[:-2])

# We only need the year and total vacancies for graphing
vacancies_df = employment_df[['Quarter', 'VALUE']].copy()
vacancies_df = vacancies_df.groupby(['Quarter'], as_index=False)['VALUE'].sum()

# Quarter has been removed, rename to year time frame
vacancies_df.columns = ['Year', 'Open Vacancies']
vacancies_df.head()

In [ ]:
# Add the job vacancies data to the already created graph with the growth and pop data
fig2.add_trace(
    go.Scatter(x=vacancies_df['Year'], y=vacancies_df['Open Vacancies'], name="Open Vacancies"),
    secondary_y=False,
)
fig2.show()

In [ ]:
# Prepare the merge df to add the job vacancies data (only 2008 onwards is available in vacancies)
merge_df = merge_df[merge_df['Year'] >= 2008]

# Cast the Year columns as a Int for merge
vacancies_df['Year'] = pd.to_numeric(vacancies_df['Year'])

# Add the vacancies df with merge
merge_df = pd.merge(merge_df, vacancies_df, how='outer', on='Year')
merge_df

In [ ]:
# Appears to be a corr. between migration and job vacancies - is this true?
corrMatrix = merge_df.corr()
sns.heatmap(corrMatrix, annot=True)
plt.show()

In [ ]:
# Heatmap shows that Migration numbers a moderately correlated to Open Job Vacancies
# and that Population growth as a birth/death ratio has been on the decline since 2008
# TODO: ML to show projections

In [ ]:
# Install the PROPHET dependancies for future series forcasting
!pip install pystan
!pip install fbprophet

In [ ]:
from fbprophet import Prophet

In [ ]:
# Documentation shows that the time series data must be in DATETIME format. I will convert that here.
merge_df['Year'] = pd.to_datetime(merge_df['Year'], format='%Y')
merge_df.head()

In [ ]:
# The PROPHET documentation shows that the DF needs to only have 2 columns named ['ds' and 'y']
prophet_df = merge_df.drop(['Migration Delta', 'Open Vacancies'], axis=1)
prophet_df.columns = ['ds', 'y']
prophet_df.head()

In [ ]:
# train the model
m = Prophet(interval_width=0.95)
model = m.fit(prophet_df)

In [ ]:
future = m.make_future_dataframe(periods=10, freq='Y')
forcast = m.predict(future)
forcast.head()

In [ ]:
forcast.tail()

In [ ]:
fig4 = m.plot(forcast)

The forecasting chart above shows that if the population trend continues on its current trajectory, we could expect to see natural population growth to go negative, shortly after 2029. This means that Ireland is going to increasingly rely on immigration in order to support its economy. This will have a big effect on pension payments, tax collection, etc.

In [88]:
births_df = pd.read_csv("/content/drive/MyDrive/BirthsByCounty1985_2020.csv")
births_df.head()

,Year,County,All Births (Number),First Births (Number),Births within Marriage (Number),Births outside Marriage (Number),Average Age of Mother (Years),Average Age of First Time Mother (Years)
0,1985,ROI Total,62388.0,NaN,57106.0,5282.0,29.3,26.0
1,1985,Dublin City,6645.0,NaN,5132.0,1513.0,28.0,NaN
2,1985,Fingal,NaN,NaN,NaN,NaN,NaN,NaN
3,1985,Dun Laoghaire Rathdown,878.0,NaN,754.0,124.0,29.8,NaN
4,1985,South Dublin,10123.0,NaN,9280.0,843.0,29.3,NaN


In [89]:
births_df['County'].unique()

array(['ROI Total', 'Dublin City', 'Fingal', 'Dun Laoghaire Rathdown',
       'South Dublin', 'Wicklow', 'Carlow', 'Kildare', 'Kilkenny',
       'Laois', 'Longford', 'Louth', 'Meath', 'Offaly', 'Westmeath',
       'Wexford', 'Waterford County', 'Waterford City', 'Clare',
       'Cork City', 'Cork County', 'Kerry', 'Limerick City',
       'Limerick County', 'North Tipperary', 'South Tipperary',
       'Tipperary', 'Galway City', 'Leitrim', 'Mayo', 'Roscommon',
       'Sligo', 'Galway County', 'Cavan', 'Donegal', 'Monaghan',
       'Outside the ROI Total'], dtype=object)

In [90]:
# This function will tidy up the county names based on the available values in the original file
# It should remove all the duplication 'city' classifications and roll the 
def county_mapping(county):
  if 'Dublin' in county or 'Fingal' in county or 'Laoghaire' in county:
    return 'Dublin'
  elif 'Cork' in county:
    return 'Cork'
  elif 'Galway' in county:
    return 'Galway'
  elif 'Limerick' in county:
    return 'Limerick'
  elif 'Tipperary' in county:
    return 'Tipperary'
  elif 'Waterford' in county:
    return 'Waterford'
  else:
    return county

In [91]:
births_df.dropna(axis=0, how='any', subset=['All Births (Number)'], inplace=True)
births_df.head()

,Year,County,All Births (Number),First Births (Number),Births within Marriage (Number),Births outside Marriage (Number),Average Age of Mother (Years),Average Age of First Time Mother (Years)
0,1985,ROI Total,62388.0,NaN,57106.0,5282.0,29.3,26.0
1,1985,Dublin City,6645.0,NaN,5132.0,1513.0,28.0,NaN
3,1985,Dun Laoghaire Rathdown,878.0,NaN,754.0,124.0,29.8,NaN
4,1985,South Dublin,10123.0,NaN,9280.0,843.0,29.3,NaN
5,1985,Wicklow,1829.0,NaN,1634.0,195.0,28.9,NaN


In [92]:
# We need to clean up the COUNTY column to match values that are inline with the currenty county structure in Ireland
# We can start by dropping the ROI totals
births_df = births_df[births_df.County != 'ROI Total']

# We can use lambda to go through each cell in COUNTY to map to the new county name
births_df['Current_County'] = births_df['County'].apply(lambda x: county_mapping(str(x)))
births_df.head()

,Year,County,All Births (Number),First Births (Number),Births within Marriage (Number),Births outside Marriage (Number),Average Age of Mother (Years),Average Age of First Time Mother (Years),Current_County
1,1985,Dublin City,6645.0,NaN,5132.0,1513.0,28.0,NaN,Dublin
3,1985,Dun Laoghaire Rathdown,878.0,NaN,754.0,124.0,29.8,NaN,Dublin
4,1985,South Dublin,10123.0,NaN,9280.0,843.0,29.3,NaN,Dublin
5,1985,Wicklow,1829.0,NaN,1634.0,195.0,28.9,NaN,Wicklow
6,1985,Carlow,832.0,NaN,773.0,59.0,28.8,NaN,Carlow


In [93]:
# Time to map map the county data
from ipyleaflet import Map, Marker, basemaps, basemap_to_tiles

# Below block is only required to display code within Colab
from google.colab import output
output.enable_custom_widget_manager()

# Create the map aspects.
center = [53.128, -7.5]
zoom = 6

# Create a map with the defined aspects.
m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=zoom)
m

Map(center=[53.128, -7.5], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

In [94]:
# The map data needs to be read in as a JSON file to work with the GeoJSON library
import json
map_json = pd.read_json('/content/drive/MyDrive/Ireland.json')

with open('/content/drive/MyDrive/Ireland.json') as json_file:
    map_data = json.load(json_file)

In [100]:
# Here we need to create a function that will output a dictionary with the float density values for 
# each county by year with the Births dataframe

# Within this function, we can filter the dataframe by year and then 
# cast each element as a list to smash together in a dictionary to create the map layers
def births_density(year):
  try:
    # This sexy line will create a new DF by:
    # 1 filter the births_df by year
    # 2 group the counties together (when we cleaned up the city values it created some duplicate counties)
    # 3 combine the duplicate counties and sum the total ALL BIRTHS
    selected_year_df = births_df[births_df['Year'] == year].groupby(['Current_County'], as_index=False)['All Births (Number)'].sum()
    
    # Create the necessary columns to a list and return them smashed together in a dict
    county_list = selected_year_df['Current_County'].tolist()
    num_of_births_list = selected_year_df['All Births (Number)'].tolist()
    return dict(zip(county_list, num_of_births_list))
  except:
    return print("Unable to derive the correct YEAR and create a dict from values")

# Test
#x = births_density(2000)
#print(x)

{'Carlow': 706.0, 'Cavan': 714.0, 'Clare': 1372.0, 'Cork': 6064.0, 'Donegal': 1772.0, 'Dublin': 16392.0, 'Galway': 2780.0, 'Kerry': 1594.0, 'Kildare': 2881.0, 'Kilkenny': 1047.0, 'Laois': 782.0, 'Leitrim': 295.0, 'Limerick': 2406.0, 'Longford': 441.0, 'Louth': 1535.0, 'Mayo': 1438.0, 'Meath': 1921.0, 'Monaghan': 627.0, 'Offaly': 815.0, 'Roscommon': 500.0, 'Sligo': 734.0, 'Tipperary': 1993.0, 'Waterford': 1423.0, 'Westmeath': 1117.0, 'Wexford': 1616.0, 'Wicklow': 1824.0}


In [102]:
# Create the Choropleth to add the shading over the counties
import ipyleaflet
layer = ipyleaflet.Choropleth(
    geo_data=map_data,
    choro_data=x
    )

In [103]:
# Add the Choropleth layer to the map and show
m.add_layer(layer)
m

Map(bottom=5527.0, center=[53.128, -7.5], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_…

In [ ]:
# TODO: Add the sliders